In [1]:
import sys
import datetime

sys.path.append("..")

from utils.get_settings import parse

settings = parse("../utils")

settings

{'train': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv',
  'pickle_path': '../temp_result/train_data.pkl',
  'pickle_path_augument': '../temp_result/train_data_a.pkl'},
 'test': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv',
  'pickle_path': '../temp_result/test_data.pkl',
  'pickle_path_augument': '../temp_result/test_data_a.pkl'},
 'valid': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/valid_data.pkl',
  'pickle_path_augument': '../temp_result/valid_data_a.pkl'},
 'stage2': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/stage2_data.pkl'}}

In [2]:
model_name = "BIDAF_1024_baseline_2class"

In [3]:
import pandas as pd
import numpy as np
from joblib import dump, load

np.random.seed(seed=0)

import torch
torch.manual_seed(0)

In [4]:
train_data = pd.read_pickle(settings["train"]["pickle_path"])
valid_data = pd.read_pickle(settings["valid"]["pickle_path"])

train_data = pd.concat([train_data,valid_data],axis = 0)

test_data = pd.read_pickle(settings["test"]["pickle_path"])

In [5]:
columns = ['vector_bert_1024', 'pron_vector_bert_1024_mean','A_idx_bert','B_idx_bert','label','pron_idx_bert','neither_idx_bert','name_idx_bert']

In [6]:
X_train = train_data[columns]
y_train = np.array(train_data.label)


X_test = test_data[columns]
y_test = np.array(test_data.label)

In [7]:
from sklearn.model_selection import KFold 
from bidaf_2class import BIDAF
from sklearn.metrics import log_loss

score = []
pred_bidaf_tr = np.zeros((X_train.shape[0],2))

kf = KFold(n_splits=5) 
for train_index, test_index in kf.split(X_train):
    X_tr, X_ts = X_train.iloc[train_index], X_train.iloc[test_index,:]
    y_tr, y_ts = y_train[train_index], y_train[test_index]
    X_tr = X_tr[y_tr!=2]
    y_tr = y_tr[y_tr!=2]
    bidaf = BIDAF().fit(X_tr, y_tr)
    pred_ts = bidaf.predict_proba(X_ts)
    s = log_loss(y_ts[y_ts!=2],pred_ts[y_ts!=2][:,[0,1]])
    score.append(s)
    pred_bidaf_tr[test_index] = pred_ts

    

[BIDAF][BIDAF][BIDAF][BIDAF][BIDAF]

In [8]:
score

[0.3299264548903294,
 0.30912147116861366,
 0.28266037068811,
 0.3202737879706349,
 0.32148718879191945]

In [9]:
np.array(score).mean()

0.31269385470192146

In [10]:
bidaf = BIDAF().fit(X_train[y_train!=2], y_train[y_train!=2])

pred_bidaf = bidaf.predict_proba(X_test)

dump((pred_bidaf_tr,pred_bidaf), "../ensemble/"+ model_name +'+basic.joblib') 

[BIDAF]

['../ensemble/BIDAF_1024_baseline_2class+basic.joblib']

In [11]:
log_loss(y_test[y_test!=2],pred_bidaf[y_test!=2][:,[0,1]])

0.29595431471045125

In [12]:
X_train = pd.concat([X_train,X_test],axis = 0)
y_train = np.concatenate([y_train,y_test],axis = 0)

bidaf = BIDAF().fit(X_train[y_train!=2], y_train[y_train!=2])

[BIDAF]

In [13]:
dump(bidaf, model_name +'.joblib') 

['BIDAF_1024_baseline_2class.joblib']

-------------------------------------------------------------------------------------------

In [1]:
import sys
import datetime

sys.path.append("..")

from utils.get_settings import parse

settings = parse("../utils")

settings

{'train': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-test.tsv',
  'pickle_path': '../temp_result/train_data.pkl',
  'pickle_path_augument': '../temp_result/train_data_a.pkl'},
 'test': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-development.tsv',
  'pickle_path': '../temp_result/test_data.pkl',
  'pickle_path_augument': '../temp_result/test_data_a.pkl'},
 'valid': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/valid_data.pkl',
  'pickle_path_augument': '../temp_result/valid_data_a.pkl'},
 'stage2': {'file_path': 'https://raw.githubusercontent.com/google-research-datasets/gap-coreference/master/gap-validation.tsv',
  'pickle_path': '../temp_result/stage2_data.pkl'}}

In [2]:
model_name = "BIDAF_1024_baseline_2class"
import joblib
import pandas as pd
import numpy as np

bidaf = joblib.load(model_name +'.joblib')

stage2_data = pd.read_pickle(settings["stage2"]["pickle_path"])

columns = ['vector_bert_1024', 'pron_vector_bert_1024_mean','A_idx_bert',
           'B_idx_bert','label','pron_idx_bert','neither_idx_bert','name_idx_bert']
columns = [col for col in columns if col!='label']

X_test = stage2_data[columns]
X_test["label"] = 0

pred_bidaf= bidaf.predict_proba(X_test)

joblib.dump(pred_bidaf, "../ensemble/"+ model_name +'+stage2.joblib')

/home/bao/anaconda3/envs/EPFL/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


['../ensemble/BIDAF_1024_baseline_2class+stage2.joblib']

## Viewed by Xingce BAO @4.10